In [ ]:
import random                  # Отображение на графике
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import time

class Organism:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Prey(Organism):
    pass

class Predator(Organism):
    pass

class Environment:
    def __init__(self, size=100, num_prey=100, num_predators=1):
        self.size = size
        self.num_prey = num_prey
        self.num_predators = num_predators
        self.steps = 0
        self.prey = [Prey(random.randint(0, size), random.randint(0, size)) for _ in range(num_prey)]
        self.predators = [Predator(random.randint(0, size), random.randint(0, size)) for _ in range(num_predators)]
        plt.style.use('ggplot')  # Меняем стиль графика
        self.fig, self.ax = plt.subplots(figsize=(15, 9))

    def update(self):
        # Обновляем хищников
        for predator in self.predators[:]:
            x_direction = random.choice([1, -1])
            x_distance = random.choice([0, 1, 2, 3, 4])
            dx = x_direction * x_distance
            y_direction = random.choice([1, -1])
            y_distance = random.choice([0, 1, 2])
            dy = y_direction * y_distance
            predator.x = max(0, min(self.size - 1, predator.x + dx))
            predator.y = max(0, min(self.size - 1, predator.y + dy))
        # Обновляем добычу
        for prey in self.prey[:]:
            x_direction = random.choice([1, -1])
            x_distance = random.choice([0, 1, 2, 3, 4])
            dx = x_direction * x_distance
            y_direction = random.choice([1, -1])
            y_distance = random.choice([0, 1, 2])
            dy = y_direction * y_distance
            prey.x = max(0, min(self.size - 1, prey.x + dx))
            prey.y = max(0, min(self.size - 1, prey.y + dy))
            # Проверяем соприкосновение хищника и добычи
            for predator in self.predators:
                if prey.x == predator.x and prey.y == predator.y:
                    if prey in self.prey:
                        self.prey.remove(prey)
                    self.predators.append(Predator(prey.x, prey.y))
                    break  # Выходим из цикла, чтобы не обрабатывать удаленный организм
            # Проверяем соприкосновение доброго организма и хищника
            for predator in self.predators:
                for prey in self.prey:
                    if prey.x == predator.x and prey.y == predator.y:
                        if prey in self.prey:
                            self.prey.remove(prey)
                        self.predators.append(Predator(prey.x, prey.y))
                        break  # Выходим из цикла, чтобы не обрабатывать удаленный организм
        self.steps += 1
        if len(self.prey) == 0:
            print("All organisms have turned into predators!")
            return False
        return True

    def plot(self):
        self.ax.clear()
        self.ax.set_xlim(0, self.size)
        self.ax.set_ylim(0, self.size)
        self.ax.set_title("Organism Movement")
        for predator in self.predators:
            self.ax.plot(predator.x, predator.y, 'ro')
        for prey in self.prey:
            self.ax.plot(prey.x, prey.y, 'go')
        plt.text(2, 95, f'Steps: {self.steps}', color='black', fontsize=12)
        display(self.fig)
        clear_output(wait=True)  # Очищаем старый график перед отображением нового

    def run_simulation(self):
        running = True
        while running:
            running = self.update()
            self.plot()
            time.sleep(0.1)  # Задержка между кадрами для создания анимации

# Создаем симуляцию и запускаем её
env = Environment()
env.run_simulation()


In [8]:
import pygame # Плюс Синий шар
import random
import math

# Инициализация Pygame
pygame.init()

# Определение цветов
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
GRAY = (128, 128, 128)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Определение размеров окна
WIDTH, HEIGHT = 800, 600

# Минимальное расстояние для соприкосновения точек
MIN_DISTANCE = 3

# Создание окна
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Эволюция")

# Определение класса для точек
class Point:
    def __init__(self, x, y, color, speed):
        self.x = x
        self.y = y
        self.color = color
        self.speed = speed

    def move(self):
        # Определение случайного шага для красных точек
        if self.color == RED:
            step = random.randint(1, 9)
        else:
            step = self.speed
        
        # Перемещение точки с учетом шага
        self.x += random.randint(-step, step)
        self.y += random.randint(-step, step)

        # Ограничение движения точек в пределах экрана
        self.x = max(0, min(self.x, WIDTH))
        self.y = max(0, min(self.y, HEIGHT))


# Функция для обработки столкновений точек
def handle_collisions(points, all_points):
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            distance = math.sqrt((points[i].x - points[j].x)**2 + (points[i].y - points[j].y)**2)
            if distance < MIN_DISTANCE:
                if points[i].color == RED and points[j].color == GREEN:
                    points[j].color = YELLOW
                elif points[i].color == GREEN and points[j].color == YELLOW:
                    points[j].color = GREEN  # Изменение цвета желтой точки на зеленый
                    # появление Синей точки с вероятностью Х %
                    if random.random() <= 0.12:
                        all_points.append(Point(points[j].x, points[j].y, BLUE, random.randint(1, 12)))
                elif points[i].color == YELLOW and points[j].color == GREEN:
                    points[i].color = GREEN
                    # Добавление новой зеленой точки со скоростью красной
                    all_points.append(Point(points[j].x, points[j].y, GREEN, random.randint(1, 15)))
                elif points[i].color == RED and points[j].color == YELLOW:
                    points[j].color = RED
                elif points[i].color == YELLOW and points[j].color == RED:
                    points[i].color = RED
                elif points[i].color == RED and points[j].color == BLUE:
                    points[i].color = YELLOW
                elif points[i].color == BLUE and points[j].color == RED:
                    points[j].color = YELLOW
                    
         
  
                    
#Функция для отрисовки точек
def draw_points(points):
    for point in points:
        pygame.draw.circle(screen, point.color, (point.x, point.y), 5)

# Функция для отображения счетчика
def display_counter(red_count, green_count, yellow_count, blue_count, step_count):
    font = pygame.font.Font(None, 36)
    text = font.render(f"Red: {red_count}  Green: {green_count}  Yellow: {yellow_count} Blue: {blue_count}  Steps: {step_count}", True, BLACK)
    screen.blit(text, (10, 10))

# Основной игровой цикл
def main():
    running = True
    clock = pygame.time.Clock()
    step_counter = 0  # Счетчик шагов

    # Создание начальных точек
    num_red = int(input("Введите количество красных точек: "))
    num_green = int(input("Введите количество зеленых точек: "))

    red_points = [Point(random.randint(0, WIDTH), random.randint(0, HEIGHT), RED, random.randint(1, 9)) for _ in range(num_red)]
    green_points = [Point(random.randint(0, WIDTH), random.randint(0, HEIGHT), GREEN, random.randint(1, 5)) for _ in range(num_green)]
    all_points = red_points + green_points

    while running:
        screen.fill(GRAY)

        red_count = sum(1 for point in all_points if point.color == RED)
        green_count = sum(1 for point in all_points if point.color == GREEN)
        yellow_count = sum(1 for point in all_points if point.color == YELLOW)
        blue_count = sum(1 for point in all_points if point.color == BLUE)

        display_counter(red_count, green_count, yellow_count, blue_count, step_counter)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Перемещение точек
        for point in all_points:
            point.move()

        # Обработка столкновений
        handle_collisions(all_points, all_points)

        # Отрисовка точек
        draw_points(all_points)

        pygame.display.flip()
        clock.tick(60)
        step_counter += 1  # Увеличиваем счетчик шагов

    print(f"Общее количество шагов: {step_counter}")
    pygame.quit()

if __name__ == "__main__":
    main()


Введите количество красных точек: 1
Введите количество зеленых точек: 10
Общее количество шагов: 24734
